In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)



In [2]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

b = a.flatten()
b
c = a.flatten('F')
c

array([[1, 2, 3],
       [4, 5, 6]])

array([1, 2, 3, 4, 5, 6])

array([1, 4, 2, 5, 3, 6])

In [3]:
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_name, drivers_by_id

# DriverSet = OrderedSet(
#     [
#         'wang-cheng',
#         'li-changlong',
#         'chen-hongmei',
#         'zheng-longfei',
#     ]
# )
# DriverSet[0]
# DriverSet.get_loc('wang-cheng')
# DriverSet


Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): <class 'eos.data_io.struct.eos_data.RecordDoc'>; Concrete Class: (eos.data_io.pool.pool.Pool[eos.data_io.struct.eos_data.RecordDoc],)
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])


2023-06-14 09:25:08.941281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), 'H')
ts_ind
df = pd.DataFrame(a, index=ts_ind, columns=['c1', 'c2', 'c3'])
df
df.index[0]


DatetimeIndex(['2023-06-14 09:25:10.134108', '2023-06-14 10:25:10.134108'], dtype='datetime64[ns]', freq=None)

,c1,c2,c3
2023-06-14 09:25:10.134108,1,2,3
2023-06-14 10:25:10.134108,4,5,6


Timestamp('2023-06-14 09:25:10.134108')

In [5]:
df_ts = pd.DataFrame(np.array([ts]), columns=['timestamp'])
df_ts

,timestamp
0,2023-06-14 09:25:10.134108


In [6]:
s = np.arange(10)
s

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Generate state

In [7]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_s = pd.DataFrame(ss, columns=['t', 'v', 'p', 'b'])  # .set_index('t')
df_s

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]

,t,v,p,b
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [8]:
np.arange(0, 4 * 20, 20)

array([ 0, 20, 40, 60])

In [9]:
ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3}
)  # .set_index('timestep')
df_ss.columns.name = 'tuple'
df_ss


DatetimeIndex(['2023-06-14 09:25:10.134108', '2023-06-14 09:25:10.154108',
               '2023-06-14 09:25:10.174108', '2023-06-14 09:25:10.194108'],
              dtype='datetime64[ns]', freq=None)

tuple,timestep,velocity,thrust,brake
0,2023-06-14 09:25:10.134108,0,4,8
1,2023-06-14 09:25:10.154108,1,5,9
2,2023-06-14 09:25:10.174108,2,6,10
3,2023-06-14 09:25:10.194108,3,7,11


In [10]:
state = df_ss.stack().swaplevel(0, 1)
state.name = 'state'
state.index.names = ['rows', 'idx']
state.sort_index(inplace=True)
state

rows      idx
brake     0                               8
          1                               9
          2                              10
          3                              11
thrust    0                               4
          1                               5
          2                               6
          3                               7
timestep  0      2023-06-14 09:25:10.134108
          1      2023-06-14 09:25:10.154108
          2      2023-06-14 09:25:10.174108
          3      2023-06-14 09:25:10.194108
velocity  0                               0
          1                               1
          2                               2
          3                               3
Name: state, dtype: object

In [11]:
# from functools import reduce
#
#
# velocity = pd.Series(ss[:4], name='velocity')
# thrust = pd.Series(ss[4:8], name='thrust')
# brake = pd.Series(ss[8:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)


In [12]:
# from functools import reduce
#
#
# velocity = pd.Series(s[:4], name='velocity')
# thrust = pd.Series(s[4:7], name='thrust')
# brake = pd.Series(s[7:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)
#
#

### generate action

In [13]:
from functools import reduce

a = len(ss) + np.arange(15)
speed_ser = pd.Series(np.linspace(40, 60, 3), name='speed')
# row_ser
row_dict = {f'r{i}': a[i * 5 : i * 5 + 5] for i in np.arange(3)}
# row_dict
rows_df = pd.DataFrame(row_dict)
# rows_df
ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), 'ms')
ts_ser = pd.Series(ts_ind, name='timestep')
throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name='throttle')
# throttle_ser
dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
action = (
    reduce(
        lambda left, right: pd.merge(
            left, right, how='outer', left_index=True, right_index=True
        ),
        dfs,
    )
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)

action.name = 'action'
action.index.names = ['rows', 'idx']
action.index
action
# action = pd.concat([df.DataFrame(row_dict),df.Series(row_ind),df.Series(ts_ind),df.Series(pedal_ind)],axis=1)
# action
# pedal_ind
# action_ind = pd.MultiIndex.from_product([row_ind, ts_ind], names=['row', 'timestamp'])
# a
# speed_row, row_name = *row_dict
# action = (
#     pd.DataFrame(
#         {
#             **row_dict,
#             'throttle': pedal_ind,
#             'speed': [r[1] for r in row_ind],
#         }
#     )  # .set_index('pedal')
#     # .stack()
#     # .swaplevel(0, 1)
#     .sort_index()
# )
# action
# action.columns
# action.columns = pd.MultiIndex.from_arrays(
#     [action.columns, ts_ind], names=['nominalspeed', 'timestep']
# )
# action = action.stack([0, 1]).swaplevel(0, 1).swaplevel(1, 2).sort_index()
# action.name = 'action'
# # action.index.names = ['nominalspeed', 'timestamp']
# # action.index
# action

MultiIndex([(      'r0', 0),
            (      'r0', 1),
            (      'r0', 2),
            (      'r0', 3),
            (      'r0', 4),
            (      'r1', 0),
            (      'r1', 1),
            (      'r1', 2),
            (      'r1', 3),
            (      'r1', 4),
            (      'r2', 0),
            (      'r2', 1),
            (      'r2', 2),
            (      'r2', 3),
            (      'r2', 4),
            (   'speed', 0),
            (   'speed', 1),
            (   'speed', 2),
            ('throttle', 0),
            ('throttle', 1),
            ('throttle', 2),
            ('throttle', 3),
            ('throttle', 4),
            ('timestep', 0),
            ('timestep', 1),
            ('timestep', 2)],
           names=['rows', 'idx'])

rows      idx
r0        0                              12
          1                              13
          2                              14
          3                              15
          4                              16
r1        0                              17
          1                              18
          2                              19
          3                              20
          4                              21
r2        0                              22
          1                              23
          2                              24
          3                              25
          4                              26
speed     0                            40.0
          1                            50.0
          2                            60.0
throttle  0                             0.0
          1                            0.25
          2                             0.5
          3                            0.75
          4       

In [14]:
# a = len(state) + np.arange(6)
# ar0 = pd.Series(a[:2], name='row0')
# ar1 = pd.Series(a[2:4], name='row1')
# ar2 = pd.Series(a[4:], name='row2')
# series = [ar0, ar1, ar2]
# action = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# action = action.stack().swaplevel(0, 1).sort_index()
# action.name = 'action'
# action
# len(action) + len(state)

In [15]:
reward = (
    pd.DataFrame({'work': len(ss) + len(a), 'timestep': ts_ind[0]}, index=[0])
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
# reward_index = (reward.name,  ts_ind[0], 0)
reward.index.names = ['rows', 'idx']
reward.name = 'reward'
reward.index
reward
# reward.name = 'reward'

MultiIndex([('timestep', 0),
            (    'work', 0)],
           names=['rows', 'idx'])

rows      idx
timestep  0      2023-06-14 09:25:10.234108
work      0                              27
Name: reward, dtype: object

In [16]:
# velocity = pd.Series(len(state) + len(action) + len(reward) + s[:4], name='velocity')
# thrust = pd.Series(len(state) + len(action) + len(reward) + s[4:7], name='thrust')
# brake = pd.Series(len(state) + len(action) + len(reward) + s[7:], name='brake')
# series = [velocity, thrust, brake]
# nstate = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# nstate = nstate.stack().swaplevel(0, 1).sort_index()
# nstate.name = 'nstate'
# nstate
# len(nstate)
ts_ind = ts + pd.to_timedelta(5, 's') + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12) + len(ss) + len(a) + len(reward)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
nstate = (
    pd.DataFrame({'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3})
    # .set_index('timestamp')
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = 'nstate'
nstate.index.names = ['rows', 'idx']
len(nstate)
nstate

DatetimeIndex(['2023-06-14 09:25:15.134108', '2023-06-14 09:25:15.154108',
               '2023-06-14 09:25:15.174108', '2023-06-14 09:25:15.194108'],
              dtype='datetime64[ns]', freq=None)

16

rows      idx
brake     0                              37
          1                              38
          2                              39
          3                              40
thrust    0                              33
          1                              34
          2                              35
          3                              36
timestep  0      2023-06-14 09:25:15.134108
          1      2023-06-14 09:25:15.154108
          2      2023-06-14 09:25:15.174108
          3      2023-06-14 09:25:15.194108
velocity  0                              29
          1                              30
          2                              31
          3                              32
Name: nstate, dtype: object

In [17]:
timestamp = pd.Series([ts], name='timestamp')
timestamp.index = pd.MultiIndex.from_product(
    [timestamp.index, [0]], names=['rows', 'idx']
)
timestamp.index
state.index
action.index
reward.index
nstate.index

MultiIndex([(0, 0)],
           names=['rows', 'idx'])

MultiIndex([(   'brake', 0),
            (   'brake', 1),
            (   'brake', 2),
            (   'brake', 3),
            (  'thrust', 0),
            (  'thrust', 1),
            (  'thrust', 2),
            (  'thrust', 3),
            ('timestep', 0),
            ('timestep', 1),
            ('timestep', 2),
            ('timestep', 3),
            ('velocity', 0),
            ('velocity', 1),
            ('velocity', 2),
            ('velocity', 3)],
           names=['rows', 'idx'])

MultiIndex([(      'r0', 0),
            (      'r0', 1),
            (      'r0', 2),
            (      'r0', 3),
            (      'r0', 4),
            (      'r1', 0),
            (      'r1', 1),
            (      'r1', 2),
            (      'r1', 3),
            (      'r1', 4),
            (      'r2', 0),
            (      'r2', 1),
            (      'r2', 2),
            (      'r2', 3),
            (      'r2', 4),
            (   'speed', 0),
            (   'speed', 1),
            (   'speed', 2),
            ('throttle', 0),
            ('throttle', 1),
            ('throttle', 2),
            ('throttle', 3),
            ('throttle', 4),
            ('timestep', 0),
            ('timestep', 1),
            ('timestep', 2)],
           names=['rows', 'idx'])

MultiIndex([('timestep', 0),
            (    'work', 0)],
           names=['rows', 'idx'])

MultiIndex([(   'brake', 0),
            (   'brake', 1),
            (   'brake', 2),
            (   'brake', 3),
            (  'thrust', 0),
            (  'thrust', 1),
            (  'thrust', 2),
            (  'thrust', 3),
            ('timestep', 0),
            ('timestep', 1),
            ('timestep', 2),
            ('timestep', 3),
            ('velocity', 0),
            ('velocity', 1),
            ('velocity', 2),
            ('velocity', 3)],
           names=['rows', 'idx'])

In [18]:
timestamp_index = (timestamp.name, '', 0)
# timestamp_index
state_index = [(state.name, *i) for i in state.index]
# state_index
reward_index = [(reward.name, *i) for i in reward.index]
# reward_index
action_index = [(action.name, *i) for i in action.index]
# action_index
nstate_index = [(nstate.name, *i) for i in nstate.index]
# nstate_index

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
)
multiindex


MultiIndex([('timestamp',         '', 0),
            (    'state',    'brake', 0),
            (    'state',    'brake', 1),
            (    'state',    'brake', 2),
            (    'state',    'brake', 3),
            (    'state',   'thrust', 0),
            (    'state',   'thrust', 1),
            (    'state',   'thrust', 2),
            (    'state',   'thrust', 3),
            (    'state', 'timestep', 0),
            (    'state', 'timestep', 1),
            (    'state', 'timestep', 2),
            (    'state', 'timestep', 3),
            (    'state', 'velocity', 0),
            (    'state', 'velocity', 1),
            (    'state', 'velocity', 2),
            (    'state', 'velocity', 3),
            (   'action',       'r0', 0),
            (   'action',       'r0', 1),
            (   'action',       'r0', 2),
            (   'action',       'r0', 3),
            (   'action',       'r0', 4),
            (   'action',       'r1', 0),
            (   'action',       'r

In [19]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
# observation.dtype

timestamp            0    2023-06-14 09:25:10.134108
state      brake     0                             8
                     1                             9
                     2                            10
                     3                            11
                                     ...            
nstate     timestep  3    2023-06-14 09:25:15.194108
           velocity  0                            29
                     1                            30
                     2                            31
                     3                            32
Length: 61, dtype: object

In [20]:

observation0 = observation.copy()
observation0.loc['timestamp', '', 0] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]


In [21]:

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ['tuple', 'rows', 'idx']
dfs_epi.columns
dfs_epi


MultiIndex([('timestamp',         '', 0),
            (    'state',    'brake', 0),
            (    'state',    'brake', 1),
            (    'state',    'brake', 2),
            (    'state',    'brake', 3),
            (    'state',   'thrust', 0),
            (    'state',   'thrust', 1),
            (    'state',   'thrust', 2),
            (    'state',   'thrust', 3),
            (    'state', 'timestep', 0),
            (    'state', 'timestep', 1),
            (    'state', 'timestep', 2),
            (    'state', 'timestep', 3),
            (    'state', 'velocity', 0),
            (    'state', 'velocity', 1),
            (    'state', 'velocity', 2),
            (    'state', 'velocity', 3),
            (   'action',       'r0', 0),
            (   'action',       'r0', 1),
            (   'action',       'r0', 2),
            (   'action',       'r0', 3),
            (   'action',       'r0', 4),
            (   'action',       'r1', 0),
            (   'action',       'r

tuple                  timestamp state                              
rows                             brake            thrust            
idx                            0     0  1   2   3      0  1  2  3   
0     2023-06-14 10:25:10.134108     8  9  10  11      4  5  6  7  \
1     2023-06-14 11:25:10.134108     8  9  10  11      4  5  6  7   
2     2023-06-14 12:25:10.134108     8  9  10  11      4  5  6  7   
3     2023-06-14 13:25:10.134108     8  9  10  11      4  5  6  7   
4     2023-06-14 14:25:10.134108     8  9  10  11      4  5  6  7   

tuple                             ... nstate                                  
rows                    timestep  ... thrust                       timestep   
idx                            0  ...      2   3                          0   
0     2023-06-14 09:25:10.134108  ...     35  36 2023-06-14 09:25:15.134108  \
1     2023-06-14 09:25:10.134108  ...     35  36 2023-06-14 09:25:15.134108   
2     2023-06-14 09:25:10.134108  ...     35  36 2023-06-14 09:25:15.134108   
3     2023-06-14 09:25:10.134108  ...     35  36 2023-06-14 09:25:15.134108   
4     2023-06-14 09:25:10.134108  ...     35  36 2023-06-14 09:25:15.134108   

tuple                                                         
rows                                                          
idx                            1                          2   
0     2023-06-14 09:25:15.154108 2023-06-14 09:25:15.174108  \
1     2023-06-14 09:25:15.154108 2023-06-14 09:25:15.174108   
2     2023-06-14 09:25:15.154108 2023-06-14 09:25:15.174108   
3     2023-06-14 09:25:15.154108 2023-06-14 09:25:15.174108   
4     2023-06-14 09:25:15.154108 2023-06-14 09:25:15.174108   

tuple                                                  
rows                             velocity              
idx                            3        0   1   2   3  
0     2023-06-14 09:25:15.194108       29  30  31  32  
1     2023-06-14 09:25:15.194108       29  30  31  32  
2     2023-06-14 09:25:15.194108       29  30  31  32  
3     2023-06-14 09:25:15.194108       29  30  31  32  
4     2023-06-14 09:25:15.194108       29  30  31  32  

[5 rows x 61 columns]

In [22]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(('timestamp', '', 0), inplace=True)
dfs_episode1.index.name = 'timestamp'
idx = pd.IndexSlice
dfs_episode1

tuple                      action                                      ...   
rows                           r0                  r1                  ...   
idx                             0   1   2   3   4   0   1   2   3   4  ...   
timestamp                                                              ...   
2023-06-14 10:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...  \
2023-06-14 11:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 12:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 13:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 14:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   

tuple                       state                                 
rows                       thrust                      timestep   
idx                             2  3                          0   
timestamp                                                         
2023-06-14 10:25:10.134108      6  7 2023-06-14 09:25:10.134108  \
2023-06-14 11:25:10.134108      6  7 2023-06-14 09:25:10.134108   
2023-06-14 12:25:10.134108      6  7 2023-06-14 09:25:10.134108   
2023-06-14 13:25:10.134108      6  7 2023-06-14 09:25:10.134108   
2023-06-14 14:25:10.134108      6  7 2023-06-14 09:25:10.134108   

tuple                                                   
rows                                                    
idx                                                 1   
timestamp                                               
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.154108   

tuple                                                   
rows                                                    
idx                                                 2   
timestamp                                               
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.174108  \
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.174108   

tuple                                                                    
rows                                                  velocity           
idx                                                 3        0  1  2  3  
timestamp                                                                
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.194108        0  1  2  3  
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.194108        0  1  2  3  
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.194108        0  1  2  3  
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.194108        0  1  2  3  
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.194108        0  1  2  3  

[5 rows x 60 columns]

In [23]:
dfs_episode1.loc[:, idx['action':'state', :, :, :]] = dfs_episode1.loc[
    :, idx['action':'state', :, :, :]
].astype('int')
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

tuple                      action                                      ...   
rows                           r0                  r1                  ...   
idx                             0   1   2   3   4   0   1   2   3   4  ...   
timestamp                                                              ...   
2023-06-14 10:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...  \
2023-06-14 11:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 12:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 13:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   
2023-06-14 14:25:10.134108     12  13  14  15  16  17  18  19  20  21  ...   

tuple                       state                           
rows                       thrust                timestep   
idx                             2  3                    0   
timestamp                                                   
2023-06-14 10:25:10.134108      6  7  1686734710134108000  \
2023-06-14 11:25:10.134108      6  7  1686734710134108000   
2023-06-14 12:25:10.134108      6  7  1686734710134108000   
2023-06-14 13:25:10.134108      6  7  1686734710134108000   
2023-06-14 14:25:10.134108      6  7  1686734710134108000   

tuple                                                                  
rows                                                                   
idx                                           1                    2   
timestamp                                                              
2023-06-14 10:25:10.134108  1686734710154108000  1686734710174108000  \
2023-06-14 11:25:10.134108  1686734710154108000  1686734710174108000   
2023-06-14 12:25:10.134108  1686734710154108000  1686734710174108000   
2023-06-14 13:25:10.134108  1686734710154108000  1686734710174108000   
2023-06-14 14:25:10.134108  1686734710154108000  1686734710174108000   

tuple                                                              
rows                                            velocity           
idx                                           3        0  1  2  3  
timestamp                                                          
2023-06-14 10:25:10.134108  1686734710194108000        0  1  2  3  
2023-06-14 11:25:10.134108  1686734710194108000        0  1  2  3  
2023-06-14 12:25:10.134108  1686734710194108000        0  1  2  3  
2023-06-14 13:25:10.134108  1686734710194108000        0  1  2  3  
2023-06-14 14:25:10.134108  1686734710194108000        0  1  2  3  

[5 rows x 60 columns]

tuple   rows      idx
action  r0        0      object
                  1      object
                  2      object
                  3      object
                  4      object
        r1        0      object
                  1      object
                  2      object
                  3      object
                  4      object
        r2        0      object
                  1      object
                  2      object
                  3      object
                  4      object
        speed     0      object
                  1      object
                  2      object
        throttle  0      object
                  1      object
                  2      object
                  3      object
                  4      object
        timestep  0       int64
                  1       int64
                  2       int64
nstate  brake     0      object
                  1      object
                  2      object
                  3      object
        thrust    

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

### Convert columns types to and float

In [24]:
dfs_episode = dfs_epi.copy().sort_index(axis=1)
dfs_episode.set_index(('timestamp', '', 0), inplace=True)
dfs_episode.index.name = 'timestamp'
idx = pd.IndexSlice

# # dfs_episode = dfs_episode.astype({idx['state', 'velocity', :]: 'int'})  # Error unhashable type: 'slice' for dictionary
# # sorted_idx = **(idx['state', :, :]:'int')
# # sorted_idx
# dfs_episode.loc[:, idx['state', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
#     :, idx['state', ['brake', 'thrust', 'velocity'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# dfs_episode.loc[:, idx['action', ['r0', 'r1', 'r2'], :]] = dfs_episode.loc[
#     :, idx['action', ['r0', 'r1', 'r2'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
# # dfs_episode['action'] = dfs_episode['action'].astype('float')
# # dfs_episode['reward'] = dfs_episode['reward'].astype('float')
# dfs_episode.loc[:, idx['reward', 'work', 0]] = dfs_episode.loc[
#     :, idx['reward', 'work', 0]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# # dfs_episode['nstate'] = dfs_episode['nstate'].astype('float')
# dfs_episode.loc[:, idx['nstate', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
#     :, idx['nstate', ['brake', 'thrust', 'velocity'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
state_cols_float = [('state', col) for col in ['brake', 'thrust', 'velocity']]
action_cols_float = [('action', col) for col in ['r0', 'r1', 'r2', 'speed', 'throttle']]
reward_cols_float = [('reward', 'work')]
nstate_cols_float = [('nstate', col) for col in ['brake', 'thrust', 'velocity']]
for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
    dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
        'float'
    )  # float16 not allowed in parquet
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode


tuple                      action                                             
rows                           r0                            r1               
idx                             0     1     2     3     4     0     1     2   
timestamp                                                                     
2023-06-14 10:25:10.134108   12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0  \
2023-06-14 11:25:10.134108   12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0   
2023-06-14 12:25:10.134108   12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0   
2023-06-14 13:25:10.134108   12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0   
2023-06-14 14:25:10.134108   12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0   

tuple                                   ...  state        
rows                                    ... thrust        
idx                            3     4  ...      2    3   
timestamp                               ...               
2023-06-14 10:25:10.134108  20.0  21.0  ...    6.0  7.0  \
2023-06-14 11:25:10.134108  20.0  21.0  ...    6.0  7.0   
2023-06-14 12:25:10.134108  20.0  21.0  ...    6.0  7.0   
2023-06-14 13:25:10.134108  20.0  21.0  ...    6.0  7.0   
2023-06-14 14:25:10.134108  20.0  21.0  ...    6.0  7.0   

tuple                                                   
rows                                         timestep   
idx                                                 0   
timestamp                                               
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.134108   
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.134108   
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                   
rows                                                    
idx                                                 1   
timestamp                                               
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.154108   
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.154108   

tuple                                                   
rows                                                    
idx                                                 2   
timestamp                                               
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.174108  \
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.174108   
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.174108   

tuple                                                                          
rows                                                  velocity                 
idx                                                 3        0    1    2    3  
timestamp                                                                      
2023-06-14 10:25:10.134108 2023-06-14 09:25:10.194108      0.0  1.0  2.0  3.0  
2023-06-14 11:25:10.134108 2023-06-14 09:25:10.194108      0.0  1.0  2.0  3.0  
2023-06-14 12:25:10.134108 2023-06-14 09:25:10.194108      0.0  1.0  2.0  3.0  
2023-06-14 13:25:10.134108 2023-06-14 09:25:10.194108      0.0  1.0  2.0  3.0  
2023-06-14 14:25:10.134108 2023-06-14 09:25:10.194108      0.0  1.0  2.0  3.0  

[5 rows x 60 columns]

tuple   rows      idx
action  r0        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r1        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r2        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        speed     0             float64
                  1             float64
                  2             float64
        throttle  0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        timestep  0      datetime64[ns]
                  

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [25]:
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_name['VB7'].vid], names=['vehicle']
)

In [26]:
dfs_episode.index
dfs_episode.columns

MultiIndex([('VB7', 'wang-cheng', '2023-06-14 10:25:10.134108'),
            ('VB7', 'wang-cheng', '2023-06-14 11:25:10.134108'),
            ('VB7', 'wang-cheng', '2023-06-14 12:25:10.134108'),
            ('VB7', 'wang-cheng', '2023-06-14 13:25:10.134108'),
            ('VB7', 'wang-cheng', '2023-06-14 14:25:10.134108')],
           names=['vehicle', 'driver', 'timestamp'])

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

In [27]:

# dfs_episode.columns.set_names(
#     [
#         'tuple',
#         'rows',
#         'idx',
#     ],
#     level=[0, 1, 2],
#     inplace=True,
# )
dfs_episode
dfs_episode.dtypes


tuple                                         action                           
rows                                              r0                           
idx                                                0     1     2     3     4   
vehicle driver     timestamp                                                   
VB7     wang-cheng 2023-06-14 10:25:10.134108   12.0  13.0  14.0  15.0  16.0  \
                   2023-06-14 11:25:10.134108   12.0  13.0  14.0  15.0  16.0   
                   2023-06-14 12:25:10.134108   12.0  13.0  14.0  15.0  16.0   
                   2023-06-14 13:25:10.134108   12.0  13.0  14.0  15.0  16.0   
                   2023-06-14 14:25:10.134108   12.0  13.0  14.0  15.0  16.0   

tuple                                                                         
rows                                             r1                           
idx                                               0     1     2     3     4   
vehicle driver     timestamp                                                  
VB7     wang-cheng 2023-06-14 10:25:10.134108  17.0  18.0  19.0  20.0  21.0  \
                   2023-06-14 11:25:10.134108  17.0  18.0  19.0  20.0  21.0   
                   2023-06-14 12:25:10.134108  17.0  18.0  19.0  20.0  21.0   
                   2023-06-14 13:25:10.134108  17.0  18.0  19.0  20.0  21.0   
                   2023-06-14 14:25:10.134108  17.0  18.0  19.0  20.0  21.0   

tuple                                          ...  state        
rows                                           ... thrust        
idx                                            ...      2    3   
vehicle driver     timestamp                   ...               
VB7     wang-cheng 2023-06-14 10:25:10.134108  ...    6.0  7.0  \
                   2023-06-14 11:25:10.134108  ...    6.0  7.0   
                   2023-06-14 12:25:10.134108  ...    6.0  7.0   
                   2023-06-14 13:25:10.134108  ...    6.0  7.0   
                   2023-06-14 14:25:10.134108  ...    6.0  7.0   

tuple                                                                      
rows                                                            timestep   
idx                                                                    0   
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                   2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   
                   2023-06-14 12:25:10.134108 2023-06-14 09:25:10.134108   
                   2023-06-14 13:25:10.134108 2023-06-14 09:25:10.134108   
                   2023-06-14 14:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                                      
rows                                                                       
idx                                                                    1   
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
                   2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   
                   2023-06-14 12:25:10.134108 2023-06-14 09:25:10.154108   
                   2023-06-14 13:25:10.134108 2023-06-14 09:25:10.154108   
                   2023-06-14 14:25:10.134108 2023-06-14 09:25:10.154108   

tuple                                                                      
rows                                                                       
idx                                                                    2   
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.174108  \
                   2023-06-14 11:25:10.134108 2023-06-14 09:25:10.174108   
                   2023-06-14 12:25:10.134108 2023-06-14 09:25:10.174108   
                   2023-06-14 13:25:10.134108 2023-06-14 09:25:10.1741

tuple   rows      idx
action  r0        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r1        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r2        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        speed     0             float64
                  1             float64
                  2             float64
        throttle  0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        timestep  0      datetime64[ns]
                  

In [28]:
# dfs_episode['state', 'thrust'] = dfs_episode['state', 'thrust'].astype('float')
# # dfs_episode.loc[:, idx['state', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
# #     :, idx['state', ['brake', 'thrust', 'velocity'], :]
# # ].astype(
# #     'float'
# # )  # float16 not allowed in parquet
# dfs_episode.loc[:, ('state', ['brake', 'thrust', 'velocity'])] = dfs_episode.loc[
#     :, ('state', ['brake', 'thrust', 'velocity'])
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
# dfs_episode['action', 'r0'] = dfs_episode['action', 'r0'].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# dfs_episode.dtypes

In [29]:
# idx = pd.IndexSlice
# dfs_episode.loc[:, idx['state', :, :]] = dfs_episode.loc[:, idx['state', :, :]].astype(
#     'int'
# )
# dfs_episode.loc[:, idx['action', :, :]] = dfs_episode.loc[
#     :, idx['action', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['reward', :, :]] = dfs_episode.loc[
#     :, idx['reward', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['nstate', :, :]] = dfs_episode.loc[
#     :, idx['nstate', :, :]
# ].astype('float16')
#
# vel_1 = dfs_episode[[('state', 'velocity', 1)]]
# vel_1.dtypes
# vel_1.index
# vel_1.values
# # type(vel_1)
# vel_1.iloc[0]
# type(vel_1.iloc[0])
# type(vel_1.iloc[0].values[0])

## Add a level of index for episode

In [30]:
episodestart = ts - pd.Timedelta(1, 'h')
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])
dfs_episode.sort_index(inplace=True)
dfs_episode.index
dfs_episode.columns
dfs_episode

MultiIndex([('2023-06-14 09:25:10.134108', 'VB7', 'wang-cheng', ...),
            ('2023-06-14 09:25:10.134108', 'VB7', 'wang-cheng', ...),
            ('2023-06-14 09:25:10.134108', 'VB7', 'wang-cheng', ...),
            ('2023-06-14 09:25:10.134108', 'VB7', 'wang-cheng', ...),
            ('2023-06-14 09:25:10.134108', 'VB7', 'wang-cheng', ...)],
           names=['episodestart', 'vehicle', 'driver', 'timestamp'])

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

tuple                                                                    action   
rows                                                                         r0   
idx                                                                           0   
episodestart               vehicle driver     timestamp                           
2023-06-14 09:25:10.134108 VB7     wang-cheng 2023-06-14 10:25:10.134108   12.0  \
                                              2023-06-14 11:25:10.134108   12.0   
                                              2023-06-14 12:25:10.134108   12.0   
                                              2023-06-14 13:25:10.134108   12.0   
                                              2023-06-14 14:25:10.134108   12.0   

tuple                                                                            
rows                                                                             
idx                                                                          1   
episodestart               vehicle driver     timestamp                          
2023-06-14 09:25:10.134108 VB7     wang-cheng 2023-06-14 10:25:10.134108  13.0  \
                                              2023-06-14 11:25:10.134108  13.0   
                                              2023-06-14 12:25:10.134108  13.0   
                                              2023-06-14 13:25:10.134108  13.0   
                                              2023-06-14 14:25:10.134108  13.0   

tuple                                                                            
rows                                                                             
idx                                                                          2   
episodestart               vehicle driver     timestamp                          
2023-06-14 09:25:10.134108 VB7     wang-cheng 2023-06-14 10:25:10.134108  14.0  \
                                              2023-06-14 11:25:10.134108  14.0   
                                              2023-06-14 12:25:10.134108  14.0   
                                              2023-06-14 13:25:10.134108  14.0   
                                              2023-06-14 14:25:10.134108  14.0   

tuple                                                                            
rows                                                                             
idx                                                                          3   
episodestart               vehicle driver     timestamp                          
2023-06-14 09:25:10.134108 VB7     wang-cheng 2023-06-14 10:25:10.134108  15.0  \
                                              2023-06-14 11:25:10.134108  15.0   
                                              2023-06-14 12:25:10.134108  15.0   
                                              2023-06-14 13:25:10.134108  15.0   
                                              2023-06-14 14:25:10.134108  15.0   

tuple                                                                            
rows                                                                             
idx                                                                          4   
episodestart               vehicle driver     timestamp                          
2023-06-14 09:25:10.134108 VB7     wang-cheng 2023-06-14 10:25:10.134108  16.0  \
                                              2023-06-14 11:25:10.134108  16.0   
                                              2023-06-14 12:25:10.134108  16.0   
                                              2023-06-14 13:25:10.134108  16.0   
                                              2023-06-14 14:25:10.134108  16.0   

tuple                                                                            
rows                                                                        r1   
idx                                                                          0   
episodestart               vehicle driver     time

In [31]:
dfs_episode.dtypes

tuple   rows      idx
action  r0        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r1        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r2        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        speed     0             float64
                  1             float64
                  2             float64
        throttle  0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        timestep  0      datetime64[ns]
                  

In [32]:
ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, 'd')
episodestart

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_name['VB7'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_name['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_name['VB7'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_name['MP73'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode0



Timestamp('2023-06-14 09:25:10.591576')

Timestamp('2023-06-12 09:25:10.591576')

tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
episodestart               vehicle driver        timestamp                           
2023-06-12 09:25:10.591576 VB7     zheng-longfei 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                                                             1   
episodestart               vehicle driver        timestamp                          
2023-06-12 09:25:10.591576 VB7     zheng-longfei 2023-06-14 10:25:10.134108  13.0  \
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 13:25:10.134108  13.0   
                                                 2023-06-14 14:25:10.134108  13.0   

tuple                                                                               
rows                                                                                
idx                                                                             2   
episodestart               vehicle driver        timestamp                          
2023-06-12 09:25:10.591576 VB7     zheng-longfei 2023-06-14 10:25:10.134108  14.0  \
                                                 2023-06-14 11:25:10.134108  14.0   
                                                 2023-06-14 12:25:10.134108  14.0   
                                                 2023-06-14 13:25:10.134108  14.0   
                                                 2023-06-14 14:25:10.134108  14.0   

tuple                                                                               
rows                                                                                
idx                                                                             3   
episodestart               vehicle driver        timestamp                          
2023-06-12 09:25:10.591576 VB7     zheng-longfei 2023-06-14 10:25:10.134108  15.0  \
                                                 2023-06-14 11:25:10.134108  15.0   
                                                 2023-06-14 12:25:10.134108  15.0   
                                                 2023-06-14 13:25:10.134108  15.0   
                                                 2023-06-14 14:25:10.134108  15.0   

tuple                                                                               
rows                                                                                
idx                                                                             4   
episodestart               vehicle driver        timestamp                          
2023-06-12 09:25:10.591576 VB7     zheng-longfei 2023-06-14 10:25:10.134108  16.0  \
                                                 2023-06-14 11:25:10.134108  16.0   
                                                 2023-06-14 12:25:10.134108  16.0   
                                                 2023-06-14 13:25:10.134108  16.0   
                                                 2023-06-14 14:25:10.134108  16.0   

tuple                                                                               
rows                                                                        

In [33]:
# dfs_episode_concat = pd.concat([dfs_episode, dfs_episode0], axis=0)
# display('dfs_episode_concat')
# dfs_episode_concat.index
# dfs_episode_concat.columns

In [34]:
from functools import reduce

episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
display('dfs_episode_all')
dfs_episode_all.index
dfs_episode_all.columns

dfs_episode_all
tuple                                                                       state   
rows                                                                        brake   
idx                                                                             0   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  \
                                                 2023-06-14 11:25:10.134108   8.0   
                                                 2023-06-14 12:25:10.134108   8.0   
                                                 2023-06-14 13:25:10.134108   8.0   
                                                 2023-06-14 14:25:10.134108   8.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   8.0   
                                                 2023-06-14 11:25:10.134108   8.0   
                                                 2023-06-14 12:25:10.134108   8.0   
                                                 2023-06-14 13:25:10.134108   8.0   
                                                 2023-06-14 14:25:10.134108   8.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   8.0   
                                                 2023-06-14 11:25:10.134108   8.0   
                                                 2023-06-14 12:25:10.134108   8.0   
                                                 2023-06-14 13:25:10.134108   8.0   
                                                 2023-06-14 14:25:10.134108   8.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   8.0   
                                                 2023-06-14 11:25:10.134108   8.0   
                                                 2023-06-14 12:25:10.134108   8.0   
                                                 2023-06-14 13:25:10.134108   8.0   
                                                 2023-06-14 14:25:10.134108   8.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   8.0   
                                                 2023-06-14 11:25:10.134108   8.0   
                                                 2023-06-14 12:25:10.134108   8.0   
                                                 2023-06-14 13:25:10.134108   8.0   
                                                 2023-06-14 14:25:10.134108   8.0   

tuple                                                                              
rows                                                                               
idx                                                                            1   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  9.0  \
                                                 2023-06-14 11:25:10.134108  9.0   
                                                 2023-06-14 12:25:10.134108  9.0   
                                                 2023-06-14 13:25:10.134108  9.0   
                                                 2023-06-14 14:25:10.134108  9.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  9.0   
                                                 2023-06-14 11:25:10.134108  9.0   
                                                 2023-06-14 12:25:10.134108  9.0   
                                                 2023-06-14 13:25:10.134108  9.0   
                                                 2023-06-14 14:25:10.134108  9.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  9.0   
                                                 2023-06-14 11:25:10.134108  9.0   
                                                 2023-06-14 12:25:10.134108  9.0   
                                                 2023-06-14 13:25:10.134108  9.0   
      

MultiIndex([('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', 

MultiIndex([( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',    'brake', 3),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state',   'thrust', 3),
            ( 'state', 'timestep', 0),
            ( 'state', 'timestep', 1),
            ( 'state', 'timestep', 2),
            ( 'state', 'timestep', 3),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'velocity', 2),
            ( 'state', 'velocity', 3),
            ('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',   

## Make MultiIndex to Rows by adding levels of "Vehicle" and "Driver"

###  reset index to move vehicle and driver to columns, preprocessing for dask manipulation

In [35]:
dfs_episodes = dfs_episode_all.reset_index(
    level=['vehicle', 'driver']
)  # unstack vehicle and driver to columns in level 0 with default '' in level 1
# # dfs_episodes.columns
# # dfs_episode0.index = dfs_episode0.index.set_levels([[trucks[0].vid],[drivers[1].pid]], level=['vehicle', 'driver'], verify_integrity=False)
# old_columns = dfs_episodes.columns
# # old_columns[0]= (old_columns[0][0],0)
# new_columns_tuples = [(old_columns[0][0],0), (old_columns[1][0],0)] + old_columns[2:].to_list()
# prepend_columns = pd.MultiIndex.from_tuples(new_columns_tuples, names=old_columns.names)
# # prepend_columns

# dfs_episodes.columns = prepend_columns
dfs_episodes
# dfs_episodes.columns
# dfs_episodes.dtypes
# dfs_episode_all

tuple                                                 vehicle         driver   
rows                                                                           
idx                                                                            
episodestart               timestamp                                           
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108    MP73     wang-cheng  \
                           2023-06-14 11:25:10.134108    MP73     wang-cheng   
                           2023-06-14 12:25:10.134108    MP73     wang-cheng   
                           2023-06-14 13:25:10.134108    MP73     wang-cheng   
                           2023-06-14 14:25:10.134108    MP73     wang-cheng   
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 11:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 12:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 13:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 14:25:10.134108    MP73  zheng-longfei   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108     VB7     wang-cheng   
                           2023-06-14 11:25:10.134108     VB7     wang-cheng   
                           2023-06-14 12:25:10.134108     VB7     wang-cheng   
                           2023-06-14 13:25:10.134108     VB7     wang-cheng   
                           2023-06-14 14:25:10.134108     VB7     wang-cheng   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108     VB7     wang-cheng   
                           2023-06-14 11:25:10.134108     VB7     wang-cheng   
                           2023-06-14 12:25:10.134108     VB7     wang-cheng   
                           2023-06-14 13:25:10.134108     VB7     wang-cheng   
                           2023-06-14 14:25:10.134108     VB7     wang-cheng   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108     VB7  zheng-longfei   
                           2023-06-14 11:25:10.134108     VB7  zheng-longfei   
                           2023-06-14 12:25:10.134108     VB7  zheng-longfei   
                           2023-06-14 13:25:10.134108     VB7  zheng-longfei   
                           2023-06-14 14:25:10.134108     VB7  zheng-longfei   

tuple                                                 state                    
rows                                                  brake                    
idx                                                       0    1     2     3   
episodestart               timestamp                                           
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0  \
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 12:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 13:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 14:25:10.134108   8.0  9.0  10.0  11.0   
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 12:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 13:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 14:25:10.134108   8.0  9.0  10.0  11.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 12:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 13:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 14:25:10.134108   8.0  9.0  10.0  11.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   

In [36]:
dfs_episodes.dtypes

tuple    rows      idx
vehicle                           object
driver                            object
state    brake     0             float64
                   1             float64
                   2             float64
                               ...      
nstate   timestep  3      datetime64[ns]
         velocity  0             float64
                   1             float64
                   2             float64
                   3             float64
Length: 62, dtype: object

In [37]:
dfs_episodes.columns

MultiIndex([('vehicle',         '', ''),
            ( 'driver',         '', ''),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',    'brake',  3),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  'state',   'thrust',  3),
            (  'state', 'timestep',  0),
            (  'state', 'timestep',  1),
            (  'state', 'timestep',  2),
            (  'state', 'timestep',  3),
            (  'state', 'velocity',  0),
            (  'state', 'velocity',  1),
            (  'state', 'velocity',  2),
            (  'state', 'velocity',  3),
            ( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'a

In [38]:

dfs_episodes['vehicle'] = dfs_episodes['vehicle'].astype("category")
dfs_episodes['driver'] = dfs_episodes['driver'].astype("category")
# dfs_episodes.dtypes
# dfs_episodes.columns
# dfs_episodes[['vehicle','driver']].dtypes
# dfs_episodes[['vehicle','driver']]
# dfs_episodes[['vehicle','driver']].iloc[:3].dtypes
# sliced = dfs_episodes[['vehicle']].iloc[:3].stack(level=[-1,-2])
# type(sliced)
# sliced.index
# sliced.values

dfs_episodes_sorted = dfs_episodes.sort_index(
    axis=1, level=[0, 1, 2], ascending=[True, True, True]
)
dfs_episodes_sorted.sort_index(axis=0, inplace=True)
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=1, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=0, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted.sort_index(axis=1,inplace=True)
dfs_episodes_sorted
dfs_episodes_sorted.columns

# dfs_episodes[['driver'], ['vehicle']] = dfs_episodes[['driver', 'vehicle']].apply(lambda x: x.cat.codes)

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
             

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

## dataframe slicing

In [39]:
ts
ep_start = ts - pd.Timedelta(9, 'D')
ep_start
# time_slice
episode_slice = pd.date_range(ep_start, periods=6, freq='D')
episode_slice

Timestamp('2023-06-14 09:25:10.134108')

Timestamp('2023-06-05 09:25:10.134108')

DatetimeIndex(['2023-06-05 09:25:10.134108', '2023-06-06 09:25:10.134108',
               '2023-06-07 09:25:10.134108', '2023-06-08 09:25:10.134108',
               '2023-06-09 09:25:10.134108', '2023-06-10 09:25:10.134108'],
              dtype='datetime64[ns]', freq='D')

In [40]:
timestamp_slice = pd.date_range(ts, periods=3, freq='H') + pd.Timedelta(10, 'min')
timestamp_slice

DatetimeIndex(['2023-06-14 09:35:10.134108', '2023-06-14 10:35:10.134108',
               '2023-06-14 11:35:10.134108'],
              dtype='datetime64[ns]', freq='H')

In [41]:
dfs_episodes_sorted

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
             

In [42]:
dfs_episodes_sorted.index
dfs_episodes_sorted.columns

MultiIndex([('2023-06-07 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-09 09:25:10.5915

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

In [43]:
# dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3]]
# dfs_selected_episodes = dfs_episodes_sorted.loc[trucks[0].vid, drivers[1].pid, episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
idx = pd.IndexSlice
dfs_selected_episodes = dfs_episodes_sorted.loc[
    idx[episode_slice[0] : episode_slice[5], timestamp_slice[0] : timestamp_slice[2]],
    idx['action':'driver'],
]
dfs_selected_episodes
# dfs_selected_episodes[['action', 'reward']]

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                                        
rows                                                  speed throttle         
idx                                                       2        0     1   
episodestart               timestamp                                         
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  60.0      0.0  0.25  \
                           2023-06-14 11:25:10.134108  60.0      0.0  0.25   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  60.0      0.0  0.25   
                           2023-06-14 11:25:10.134108  60.0      0.0  0.25   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  60.0      0.0  0.25   
                           2023-06-14 11:25:10.134108  60.0      0.0  0.25   

tuple                                                                   
rows                                                                    
idx                                                      2     3    4   
episodestart               timestamp                                    
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  0.5  0.75  1.0  \
                           2023-06-14 11:25:10.134108  0.5  0.75  1.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  0.5  0.75  1.0   
                           2023-06-14 11:25:10.134108  0.5  0.75  1.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  0.5  0.75  1.0   
                           2023-06-14 11:25:10.134108  0.5  0.75  1.0   

tuple                                                                              
rows                           

In [44]:
# dfs_selected_episodes_columns = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]][['action', 'reward']]
# dfs_selected_episodes_columns

In [45]:
# dfs_episode_all_sorted = dfs_episode_all.sort_index(axis=1)
# dfs_episode_all_sorted = dfs_episode_all_sorted.sort_index(axis=0)
# dfs_episode_all_sorted

In [46]:

# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(slice('VB6'), slice('wang-cheng'), slice(None), slice(None)),:]
# idx = pd.IndexSlice
# dfs_episode_all_sorted.loc[(idx['VB6'], idx['wang-cheng','zheng-longfei'], slice(None), slice(None)),:]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns = dfs_episodes_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns

In [47]:

ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(10, 'd') - pd.Timedelta(6, 'h')
episodestart
episode_slice = pd.date_range(episodestart, periods=20, freq='d') + pd.Timedelta(
    10, 'min'
)
episode_slice
ts
time_slice = pd.date_range(ts, periods=6, freq='h') - pd.Timedelta(10, 'min')
time_slice


Timestamp('2023-06-04 03:25:10.842645')

DatetimeIndex(['2023-06-04 03:35:10.842645', '2023-06-05 03:35:10.842645',
               '2023-06-06 03:35:10.842645', '2023-06-07 03:35:10.842645',
               '2023-06-08 03:35:10.842645', '2023-06-09 03:35:10.842645',
               '2023-06-10 03:35:10.842645', '2023-06-11 03:35:10.842645',
               '2023-06-12 03:35:10.842645', '2023-06-13 03:35:10.842645',
               '2023-06-14 03:35:10.842645', '2023-06-15 03:35:10.842645',
               '2023-06-16 03:35:10.842645', '2023-06-17 03:35:10.842645',
               '2023-06-18 03:35:10.842645', '2023-06-19 03:35:10.842645',
               '2023-06-20 03:35:10.842645', '2023-06-21 03:35:10.842645',
               '2023-06-22 03:35:10.842645', '2023-06-23 03:35:10.842645'],
              dtype='datetime64[ns]', freq='D')

Timestamp('2023-06-14 09:25:10.134108')

DatetimeIndex(['2023-06-14 09:15:10.134108', '2023-06-14 10:15:10.134108',
               '2023-06-14 11:15:10.134108', '2023-06-14 12:15:10.134108',
               '2023-06-14 13:15:10.134108', '2023-06-14 14:15:10.134108'],
              dtype='datetime64[ns]', freq='H')

In [48]:

episode_slice[4]: episode_slice[8]
idx[episode_slice[4] : episode_slice[8]]
idx[episode_slice[4] : episode_slice[8], slice(None)]

slice(Timestamp('2023-06-08 03:35:10.842645'), Timestamp('2023-06-12 03:35:10.842645'), None)

(slice(Timestamp('2023-06-08 03:35:10.842645'), Timestamp('2023-06-12 03:35:10.842645'), None),
 slice(None, None, None))

In [49]:
episode_slice[4]
episode_slice[11]
# dfs_selected_episodes = dfs_episodes_sorted.loc[idx['VB7'], slice(None), episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes
# epislice = idx(episode_slice[4]:episode_slice[11])

# dfs_selected_episodes = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[11],slice(None)], idx[dfs_episodes_sorted['driver']==drivers[0].pid]]
dfs_selected_episodes1 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid
]
dfs_selected_episodes1
dfs_selected_episodes2 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes2
dfs_selected_episodes3 = dfs_episodes_sorted.loc[
    (dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid)
    & (dfs_episodes_sorted['vehicle'] == trucks_by_name['VB7'].vid)
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes3
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[8],slice(None)]].loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)]
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid) & (idx[episode_slice[4]:episode_slice[8],slice(None)])]
# dfs_selected_episodes31
# dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes


Timestamp('2023-06-08 03:35:10.842645')

Timestamp('2023-06-15 03:35:10.842645')

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 14:25:10.134108  19.0  20.0  21.0  ...   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 14:25:10.134108  19.0  20.0  21.0  ...   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 14:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                  state   
rows                                                  thrust   
idx                                                        3   
episodestart               timestamp                           
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108    7.0  \
                           2023-06-14 11:25:10.134108    7.0   
                           2023-06-14 12:25:10.134108    7.0   
                           2023-06-14 13:25:10.134108    7.0   
                           2023-06-14 14:25:10.134108    7.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108    7.0   
                           2023-06-14 1

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 14:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                  state   
rows                                                  thrust   
idx                                                        3   
episodestart               timestamp                           
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108    7.0  \
                           2023-06-14 11:25:10.134108    7.0   
                           2023-06-14 12:25:10.134108    7.0   
                           2023-06-14 13:25:10.134108    7.0   
                           2023-06-14 14:25:10.134108    7.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 12:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 13:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 14:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                                              
rows                                                                               
idx                                                                            1   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   
                          

In [50]:
idx[time_slice[0] : time_slice[3]]
dfs_selected_episodes3.columns

slice(Timestamp('2023-06-14 09:15:10.134108'), Timestamp('2023-06-14 12:15:10.134108'), None)

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

In [51]:
time_slice2 = pd.date_range(ts, periods=6, freq='S')
time_slice2
dfs_selected_episodes3_sorted = dfs_selected_episodes3.sort_index(axis=0).sort_index(
    axis=1
)[['action']]
dfs_selected_episodes3_sorted
# dfs_selected_episodes3.loc[:, idx[:, :, 0:2, :]]
# dfs_selected_episodes3_sorted.loc[:, idx[:, :,time_slice2[0]:time_slice2[1], :]]
# dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[2]],:]
dfs_selected_episodes3.loc[idx[:, time_slice[0] : time_slice[3]], :]

DatetimeIndex(['2023-06-14 09:25:10.134108', '2023-06-14 09:25:11.134108',
               '2023-06-14 09:25:12.134108', '2023-06-14 09:25:13.134108',
               '2023-06-14 09:25:14.134108', '2023-06-14 09:25:15.134108'],
              dtype='datetime64[ns]', freq='S')

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 12:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 13:25:10.134108  19.0  20.0  21.0  ...   
                           2023-06-14 14:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                                        
rows                                                  speed       throttle   
idx                                                       1     2        0   
episodestart               timestamp                                         
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  50.0  60.0      0.0  \
                           2023-06-14 11:25:10.134108  50.0  60.0      0.0   
                           2023-06-14 12:25:10.134108  50.0  60.0      0.0   
                           2023-06-14 13:25:10.134108  50.0  60.0      0.0   
                           2023-06-14 14:25:10.134108  50.0  60.0      0.0   

tuple                                                                         
rows                                                                          
idx                                                       1    2     3    4   
episodestart               timestamp                                          
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  0.25  0.5  0.75  1.0  \
                           2023-06-14 11:25:10.134108  0.25  0.5  0.75  1.0   
                           2023-06-14 12:25:10.134108  0.25  0.5  0.75  1.0   
                           2023-06-14 13:25:10.134108  0.25  0.5  0.75  1.0   
                           2023-06-14 14:25:10.134108  0.25  0.5  0.75  1.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.234108  \
                           20

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                  state   
rows                                                  thrust   
idx                                                        3   
episodestart               timestamp                           
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108    7.0  \
                           2023-06-14 11:25:10.134108    7.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                                              
rows                                                                               
idx                                                                            1   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   

tuple                                                                              
rows                                                                               
idx                                                                            2   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.174108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.174108   

tuple                                                                              
rows                                                                               
idx                                                                            3   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.194108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.194108   

tuple                                                                           
rows                                                  velocity                  
idx     

In [52]:
dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
time_slice[0]
time_slice[3]
idx[:, time_slice[0] : time_slice[3]]
dfs_selected_episodes4 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], :
]
# dfs_selected_episodes4 = dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[3],:],:]

dfs_selected_episodes4
dfs_selected_episodes41 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], 'action':'driver'
]
dfs_selected_episodes41
# dfs_selected_episodes4 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)].loc[idx[episode_slice[4]:episode_slice[8],time_slice[0]:time_slice[3]]]
# dfs_selected_episodes4

MultiIndex([('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 14:25:10.134108')],
           names=['episodestart', 'timestamp'])

Timestamp('2023-06-14 09:15:10.134108')

Timestamp('2023-06-14 12:15:10.134108')

(slice(None, None, None),
 slice(Timestamp('2023-06-14 09:15:10.134108'), Timestamp('2023-06-14 12:15:10.134108'), None))

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                  state   
rows                                                  thrust   
idx                                                        3   
episodestart               timestamp                           
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108    7.0  \
                           2023-06-14 11:25:10.134108    7.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                                              
rows                                                                               
idx                                                                            1   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.154108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.154108   

tuple                                                                              
rows                                                                               
idx                                                                            2   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.174108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.174108   

tuple                                                                              
rows                                                                               
idx                                                                            3   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.194108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.194108   

tuple                                                                           
rows                                                  velocity                  
idx     

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                                    ...   
rows                                                                     ...   
idx                                                       2     3     4  ...   
episodestart               timestamp                                     ...   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  20.0  21.0  ...  \
                           2023-06-14 11:25:10.134108  19.0  20.0  21.0  ...   

tuple                                                                        
rows                                                  speed throttle         
idx                                                       2        0     1   
episodestart               timestamp                                         
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  60.0      0.0  0.25  \
                           2023-06-14 11:25:10.134108  60.0      0.0  0.25   

tuple                                                                   
rows                                                                    
idx                                                      2     3    4   
episodestart               timestamp                                    
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  0.5  0.75  1.0  \
                           2023-06-14 11:25:10.134108  0.5  0.75  1.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.234108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.234108   

tuple                                                                              
rows                                                                               
idx                                                                            1   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.254108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.254108   

tuple                                                                              
rows                                                                               
idx                                                                            2   
episodestart               timestamp                                               
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.274108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.274108   

tuple                                                      driver  
rows                                                               
idx             

In [53]:

# dfs_selected_episodes = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns


In [54]:
dfs_episodes_sorted
# dfs_episode.loc[:,'state']
dfs_slice = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['vehicle'] == 'MP73', ['vehicle', 'driver', 'state', 'reward']
]
dfs_slice

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
             

tuple                                                 vehicle         driver   
rows                                                                           
idx                                                                            
episodestart               timestamp                                           
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108    MP73  zheng-longfei  \
                           2023-06-14 11:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 12:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 13:25:10.134108    MP73  zheng-longfei   
                           2023-06-14 14:25:10.134108    MP73  zheng-longfei   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108    MP73     wang-cheng   
                           2023-06-14 11:25:10.134108    MP73     wang-cheng   
                           2023-06-14 12:25:10.134108    MP73     wang-cheng   
                           2023-06-14 13:25:10.134108    MP73     wang-cheng   
                           2023-06-14 14:25:10.134108    MP73     wang-cheng   

tuple                                                 state                    
rows                                                  brake                    
idx                                                       0    1     2     3   
episodestart               timestamp                                           
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0  \
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 12:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 13:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 14:25:10.134108   8.0  9.0  10.0  11.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 11:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 12:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 13:25:10.134108   8.0  9.0  10.0  11.0   
                           2023-06-14 14:25:10.134108   8.0  9.0  10.0  11.0   

tuple                                                                         
rows                                                  thrust                  
idx                                                        0    1    2    3   
episodestart               timestamp                                          
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108    4.0  5.0  6.0  7.0  \
                           2023-06-14 11:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 12:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 13:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 14:25:10.134108    4.0  5.0  6.0  7.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 11:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 12:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 13:25:10.134108    4.0  5.0  6.0  7.0   
                           2023-06-14 14:25:10.134108    4.0  5.0  6.0  7.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 12:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 13:25:10.134108 2023-06-1

# Saved episodes as parquet in multi-level folders "vehicle/driver/episode.parquet"

In [55]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq
import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('pyarrow: ', pa.__version__)


Python:  3.10.9
pandas:  2.0.1
pyarrow:  11.0.0


In [56]:
os.chdir('../../data')

In [57]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

## create parquet files

In [58]:
dfs_episodes_sorted
dfs_episodes_sorted.index
dfs_episodes_sorted.columns
dfs_episodes_sorted.dtypes

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
             

MultiIndex([('2023-06-07 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-09 09:25:10.5915

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

tuple    rows      idx
action   r0        0       float64
                   1       float64
                   2       float64
                   3       float64
                   4       float64
                            ...   
state    velocity  0       float64
                   1       float64
                   2       float64
                   3       float64
vehicle                   category
Length: 62, dtype: object

In [59]:
# dfs_episode4dask = dfs_episode_all_sorted.swaplevel(0,-1, axis=0).swaplevel(1,3, axis=0).swaplevel(2,3, axis=0).unstack()
# dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# dfs_episode4dask
# # dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# # dfs_episode4dask.index
# # dfs_episode4dask.index
# # dfs_episode4dask.columns
# # dfs_episode4dask
# # dfs_episode_all_stack = dfs_episode_all.unstack(level=[-1,-2,-3,])
# # dfs_episode_all_stack

In [60]:
# dfs_episode4dask1 = dfs_episode4dask.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1)
# # dfs_episode4dask1
# dfs_episode4dask2 = dfs_episode4dask1.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1).swaplevel(-4,-5,axis=1)
# dfs_episode4dask2.sort_index(axis=1, inplace=True)
# dfs_episode4dask2


### encoding multiindex dataframe to flat index for Dask datframe

In [61]:
dfs_episodes_sorted.columns
cols = dfs_episodes_sorted.columns.to_flat_index()
cols

## 3 level index
# cols_str = [
#     f'{x[0]}_{x[1]}_{x[2]}'
#     if (x[1] != '')
#     else f'{x[0]}__{x[2]}'
#     if (x[2] != '')
#     else f'{x[0]}'
#     for x in cols
# ]

## 4 level index
cols_str = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}__'
    for x in cols
]
cols_str

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

Index([      ('action', 'r0', 0),       ('action', 'r0', 1),
             ('action', 'r0', 2),       ('action', 'r0', 3),
             ('action', 'r0', 4),       ('action', 'r1', 0),
             ('action', 'r1', 1),       ('action', 'r1', 2),
             ('action', 'r1', 3),       ('action', 'r1', 4),
             ('action', 'r2', 0),       ('action', 'r2', 1),
             ('action', 'r2', 2),       ('action', 'r2', 3),
             ('action', 'r2', 4),    ('action', 'speed', 0),
          ('action', 'speed', 1),    ('action', 'speed', 2),
       ('action', 'throttle', 0), ('action', 'throttle', 1),
       ('action', 'throttle', 2), ('action', 'throttle', 3),
       ('action', 'throttle', 4), ('action', 'timestep', 0),
       ('action', 'timestep', 1), ('action', 'timestep', 2),
              ('driver', '', ''),    ('nstate', 'brake', 0),
          ('nstate', 'brake', 1),    ('nstate', 'brake', 2),
          ('nstate', 'brake', 3),   ('nstate', 'thrust', 0),
         ('nstate', 'thr

['action_r0_0',
 'action_r0_1',
 'action_r0_2',
 'action_r0_3',
 'action_r0_4',
 'action_r1_0',
 'action_r1_1',
 'action_r1_2',
 'action_r1_3',
 'action_r1_4',
 'action_r2_0',
 'action_r2_1',
 'action_r2_2',
 'action_r2_3',
 'action_r2_4',
 'action_speed_0',
 'action_speed_1',
 'action_speed_2',
 'action_throttle_0',
 'action_throttle_1',
 'action_throttle_2',
 'action_throttle_3',
 'action_throttle_4',
 'action_timestep_0',
 'action_timestep_1',
 'action_timestep_2',
 'driver__',
 'nstate_brake_0',
 'nstate_brake_1',
 'nstate_brake_2',
 'nstate_brake_3',
 'nstate_thrust_0',
 'nstate_thrust_1',
 'nstate_thrust_2',
 'nstate_thrust_3',
 'nstate_timestep_0',
 'nstate_timestep_1',
 'nstate_timestep_2',
 'nstate_timestep_3',
 'nstate_velocity_0',
 'nstate_velocity_1',
 'nstate_velocity_2',
 'nstate_velocity_3',
 'reward_timestep_0',
 'reward_work_0',
 'state_brake_0',
 'state_brake_1',
 'state_brake_2',
 'state_brake_3',
 'state_thrust_0',
 'state_thrust_1',
 'state_thrust_2',
 'state_thrus

In [62]:
# encoding index and columns
dfs_episodes_sorted_flat_col = dfs_episodes_sorted.copy()
cols = dfs_episodes_sorted.columns.to_flat_index()
dfs_episodes_sorted_flat_col.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}__'
    for x in cols
]
dfs_episodes_sorted_flat_ind = dfs_episodes_sorted.copy()
idx = dfs_episodes_sorted.index.to_flat_index()
dfs_episodes_sorted_flat_ind.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    if x[1] != ''
    else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in idx
]
# dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat = dfs_episodes_sorted.copy()
dfs_episodes_sorted_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}__'
    for x in cols
]
dfs_episodes_sorted_flat.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    if x[1] != ''
    else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in idx
]
dfs_episodes_sorted_flat



,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-08T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7


In [63]:
dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat_col
dfs_episodes_sorted_flat
dfs_episodes_sorted_flat.dtypes

tuple                                              action                     
rows                                                   r0                     
idx                                                     0     1     2     3   
2023-06-07T09:25:10.591576_2023-06-14T10:25:10....   12.0  13.0  14.0  15.0  \
2023-06-07T09:25:10.591576_2023-06-14T11:25:10....   12.0  13.0  14.0  15.0   
2023-06-07T09:25:10.591576_2023-06-14T12:25:10....   12.0  13.0  14.0  15.0   
2023-06-07T09:25:10.591576_2023-06-14T13:25:10....   12.0  13.0  14.0  15.0   
2023-06-07T09:25:10.591576_2023-06-14T14:25:10....   12.0  13.0  14.0  15.0   
2023-06-08T09:25:10.591576_2023-06-14T10:25:10....   12.0  13.0  14.0  15.0   
2023-06-08T09:25:10.591576_2023-06-14T11:25:10....   12.0  13.0  14.0  15.0   
2023-06-08T09:25:10.591576_2023-06-14T12:25:10....   12.0  13.0  14.0  15.0   
2023-06-08T09:25:10.591576_2023-06-14T13:25:10....   12.0  13.0  14.0  15.0   
2023-06-08T09:25:10.591576_2023-06-14T14:25:10....   12.0  13.0  14.0  15.0   
2023-06-09T09:25:10.591576_2023-06-14T10:25:10....   12.0  13.0  14.0  15.0   
2023-06-09T09:25:10.591576_2023-06-14T11:25:10....   12.0  13.0  14.0  15.0   
2023-06-09T09:25:10.591576_2023-06-14T12:25:10....   12.0  13.0  14.0  15.0   
2023-06-09T09:25:10.591576_2023-06-14T13:25:10....   12.0  13.0  14.0  15.0   
2023-06-09T09:25:10.591576_2023-06-14T14:25:10....   12.0  13.0  14.0  15.0   
2023-06-12T09:25:10.591576_2023-06-14T10:25:10....   12.0  13.0  14.0  15.0   
2023-06-12T09:25:10.591576_2023-06-14T11:25:10....   12.0  13.0  14.0  15.0   
2023-06-12T09:25:10.591576_2023-06-14T12:25:10....   12.0  13.0  14.0  15.0   
2023-06-12T09:25:10.591576_2023-06-14T13:25:10....   12.0  13.0  14.0  15.0   
2023-06-12T09:25:10.591576_2023-06-14T14:25:10....   12.0  13.0  14.0  15.0   
2023-06-14T09:25:10.134108_2023-06-14T10:25:10....   12.0  13.0  14.0  15.0   
2023-06-14T09:25:10.134108_2023-06-14T11:25:10....   12.0  13.0  14.0  15.0   
2023-06-14T09:25:10.134108_2023-06-14T12:25:10....   12.0  13.0  14.0  15.0   
2023-06-14T09:25:10.134108_2023-06-14T13:25:10....   12.0  13.0  14.0  15.0   
2023-06-14T09:25:10.134108_2023-06-14T14:25:10....   12.0  13.0  14.0  15.0   

tuple                                                                        
rows                                                        r1               
idx                                                    4     0     1     2   
2023-06-07T09:25:10.591576_2023-06-14T10:25:10....  16.0  17.0  18.0  19.0  \
2023-06-07T09:25:10.591576_2023-06-14T11:25:10....  16.0  17.0  18.0  19.0   
2023-06-07T09:25:10.591576_2023-06-14T12:25:10....  16.0  17.0  18.0  19.0   
2023-06-07T09:25:10.591576_2023-06-14T13:25:10....  16.0  17.0  18.0  19.0   
2023-06-07T09:25:10.591576_2023-06-14T14:25:10....  16.0  17.0  18.0  19.0   
2023-06-08T09:25:10.591576_2023-06-14T10:25:10....  16.0  17.0  18.0  19.0   
2023-06-08T09:25:10.591576_2023-06-14T11:25:10....  16.0  17.0  18.0  19.0   
2023-06-08T09:25:10.591576_2023-06-14T12:25:10....  16.0  17.0  18.0  19.0   
2023-06-08T09:25:10.591576_2023-06-14T13:25:10....  16.0  17.0  18.0  19.0   
2023-06-08T09:25:10.591576_2023-06-14T14:25:10....  16.0  17.0  18.0  19.0   
2023-06-09T09:25:10.591576_2023-06-14T10:25:10....  16.0  17.0  18.0  19.0   
2023-06-09T09:25:10.591576_2023-06-14T11:25:10....  16.0  17.0  18.0  19.0   
2023-06-09T09:25:10.591576_2023-06-14T12:25:10....  16.0  17.0  18.0  19.0   
2023-06-09T09:25:10.591576_2023-06-14T13:25:10....  16.0  17.0  18.0  19.0   
2023-06-09T09:25:10.591576_2023-06-14T14:25:10....  16.0  17.0  18.0  19.0   
2023-06-12T09:25:10.591576_2023-06-14T10:25:10....  16.0  17.0  18.0  19.0   
2023-06-12T09:25:10.591576_2023-06-14T11:25:10....  16.0  17.0  18.0  19.0   
2023-06-12T09:25:10.591576_2023-06-14T12:25:10....  16.0  17.0  18.0  19.0   
2023-06-12T09:25:10.591576_2023-06-14T13:25:10....  16.0  17.0  18.0  19.0   
2023-06-12T09:25:10.591576_2023-06-14T14:25:10....  16.0  17.0  18.0  1

action_r0_0   
episodestart               timestamp                                 
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108         12.0  \
                           2023-06-14 11:25:10.134108         12.0   
                           2023-06-14 12:25:10.134108         12.0   
                           2023-06-14 13:25:10.134108         12.0   
                           2023-06-14 14:25:10.134108         12.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108         12.0   
                           2023-06-14 11:25:10.134108         12.0   
                           2023-06-14 12:25:10.134108         12.0   
                           2023-06-14 13:25:10.134108         12.0   
                           2023-06-14 14:25:10.134108         12.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108         12.0   
                           2023-06-14 11:25:10.134108         12.0   
                           2023-06-14 12:25:10.134108         12.0   
                           2023-06-14 13:25:10.134108         12.0   
                           2023-06-14 14:25:10.134108         12.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108         12.0   
                           2023-06-14 11:25:10.134108         12.0   
                           2023-06-14 12:25:10.134108         12.0   
                           2023-06-14 13:25:10.134108         12.0   
                           2023-06-14 14:25:10.134108         12.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108         12.0   
                           2023-06-14 11:25:10.134108         12.0   
                           2023-06-14 12:25:10.134108         12.0   
                           2023-06-14 13:25:10.134108         12.0   
                           2023-06-14 14:25:10.134108         12.0   

                                                       action_r0_1   
episodestart               timestamp                                 
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108         13.0  \
                           2023-06-14 11:25:10.134108         13.0   
                           2023-06-14 12:25:10.134108         13.0   
                           2023-06-14 13:25:10.134108         13.0   
                           2023-06-14 14:25:10.134108         13.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108         13.0   
                           2023-06-14 11:25:10.134108         13.0   
                           2023-06-14 12:25:10.134108         13.0   
                           2023-06-14 13:25:10.134108         13.0   
                           2023-06-14 14:25:10.134108         13.0   
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108         13.0   
                           2023-06-14 11:25:10.134108         13.0   
                           2023-06-14 12:25:10.134108         13.0   
                           2023-06-14 13:25:10.134108         13.0   
                           2023-06-14 14:25:10.134108         13.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108         13.0   
                           2023-06-14 11:25:10.134108         13.0   
                           2023-06-14 12:25:10.134108         13.0   
                           2023-06-14 13:25:10.134108         13.0   
                           2023-06-14 14:25:10.134108         13.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108         13.0   
                           2023-06-14 11:25:10.134108         13.0   
                           2023-06-14 12:25:10.134108         13.0   
                           2023-06-14 13:25:10.134108         13.0   
                           2023-06-14 14:25:10.134108         13.0   

                                                       action_r0_2   
episodestart               timestamp                                 
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108         14.0  \
                           2023-06-14 11:25:10.134108         1

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73
2023-06-08T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7
2023-06-08T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,VB7


action_r0_0          float64
action_r0_1          float64
action_r0_2          float64
action_r0_3          float64
action_r0_4          float64
                      ...   
state_velocity_0     float64
state_velocity_1     float64
state_velocity_2     float64
state_velocity_3     float64
vehicle__           category
Length: 62, dtype: object

In [64]:
dfs_episodes_sorted_flat['vehicle__'].iloc[0:2].values.dtype
dfs_episodes_sorted_flat['vehicle__'].dtype
dfs_episodes_sorted_flat['state_velocity_0'].dtype


CategoricalDtype(categories=['MP73', 'VB7'], ordered=False)

CategoricalDtype(categories=['MP73', 'VB7'], ordered=False)

dtype('float64')

In [65]:
try:
    ddf_episodes = dd.from_pandas(dfs_episodes_sorted_flat, npartitions=1)
except Exception as e:
    print(e)

try:
    ddf_episodes.index
    ddf_episodes.columns
except Exception as e:
    print(e)

ddf_episodes

Dask Index Structure:
npartitions=1
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108    object
2023-06-14T09:25:10.134108_2023-06-14T14:25:10.134108       ...
dtype: object
Dask Name: from_pandas-index, 2 graph layers

Index(['action_r0_0', 'action_r0_1', 'action_r0_2', 'action_r0_3',
       'action_r0_4', 'action_r1_0', 'action_r1_1', 'action_r1_2',
       'action_r1_3', 'action_r1_4', 'action_r2_0', 'action_r2_1',
       'action_r2_2', 'action_r2_3', 'action_r2_4', 'action_speed_0',
       'action_speed_1', 'action_speed_2', 'action_throttle_0',
       'action_throttle_1', 'action_throttle_2', 'action_throttle_3',
       'action_throttle_4', 'action_timestep_0', 'action_timestep_1',
       'action_timestep_2', 'driver__', 'nstate_brake_0', 'nstate_brake_1',
       'nstate_brake_2', 'nstate_brake_3', 'nstate_thrust_0',
       'nstate_thrust_1', 'nstate_thrust_2', 'nstate_thrust_3',
       'nstate_timestep_0', 'nstate_timestep_1', 'nstate_timestep_2',
       'nstate_timestep_3', 'nstate_velocity_0', 'nstate_velocity_1',
       'nstate_velocity_2', 'nstate_velocity_3', 'reward_timestep_0',
       'reward_work_0', 'state_brake_0', 'state_brake_1', 'state_brake_2',
       'state_brake_3', 'state_thrust_

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,action_r2_0,action_r2_1,action_r2_2,action_r2_3,action_r2_4,action_speed_0,action_speed_1,action_speed_2,action_throttle_0,action_throttle_1,action_throttle_2,action_throttle_3,action_throttle_4,action_timestep_0,action_timestep_1,action_timestep_2,driver__,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_brake_3,nstate_thrust_0,nstate_thrust_1,nstate_thrust_2,nstate_thrust_3,nstate_timestep_0,nstate_timestep_1,nstate_timestep_2,nstate_timestep_3,nstate_velocity_0,nstate_velocity_1,nstate_velocity_2,nstate_velocity_3,reward_timestep_0,reward_work_0,state_brake_0,state_brake_1,state_brake_2,state_brake_3,state_thrust_0,state_thrust_1,state_thrust_2,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],category[known],float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,category[known]
2023-06-14T09:25:10.134108_2023-06-14T14:25:10.134108,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [66]:

custom_metadata = {
    b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}
custom_metadata

{b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'}

In [67]:
try:
    with ProgressBar():
        ddf_episodes.to_parquet(
            'eos_dask_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

[########################################] | 100% Completed | 101.28 ms


### Read saved parquet file

In [68]:
ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.head(5)
ddf.dtypes
ddf.npartitions
ddf.divisions
ddf._meta

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__
2023-06-09T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng


action_r0_0          float64
action_r0_1          float64
action_r0_2          float64
action_r0_3          float64
action_r0_4          float64
                      ...   
state_velocity_1     float64
state_velocity_2     float64
state_velocity_3     float64
vehicle__           category
driver__            category
Length: 62, dtype: object

4

(None, None, None, None, None)

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__


### Get customized metadata with pyarrow

In [69]:

import pyarrow as pa
import pyarrow.parquet as pq
import json

# print('pyarrow: ', pa.__version__)

# table = pq.read_table('ddf_episodes')
# table_meta = table.schema.metadata[b'eos']
# print(table_meta)
# # custom_metadata_info = json.loads(table.schema.metadata[b'eos'])
# # print(custom_metadata_info)
# try:
#     custom_meta_info = json.loads(table_meta)
# except Exception as e:
#     print(f'Exception: {e}')

# print(custom_meta_info)

# # Print formatted output of the dichtionary custom_meta_info:
# for key, val in custom_meta_info.items():
#     print('{:15}: {}'.format(key, val))

# ALTERNATE:

table_meta = pq.read_metadata('eos_dask_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = table_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

# table_meta = pq.read_metadata('ddf_episodes/_metadata')
# # print(table_meta)
# # print(table_meta.metadata[b'eos'])
# custom_meta_info = table_meta.metadata[b'eos']
# custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

length: [3, 600, 5]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


### encoding flat_index of dask dataframe to multiindex

In [70]:
df_episodes_read = ddf.compute()
df_episodes_read

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__
2023-06-09T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei


In [71]:
# decoding
multi_cols = [tuple(col.split('_')) for col in df_episodes_read.columns]
multi_cols
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_cols)
# multi_idx
multi_idx.names = ['tuple', 'rows', 'idx']
driver_vehicle = [idx for idx in multi_idx if idx[0] == 'driver' or idx[0] == 'vehicle']
driver_vehicle
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_idx

[('action', 'r0', '0'),
 ('action', 'r0', '1'),
 ('action', 'r0', '2'),
 ('action', 'r0', '3'),
 ('action', 'r0', '4'),
 ('action', 'r1', '0'),
 ('action', 'r1', '1'),
 ('action', 'r1', '2'),
 ('action', 'r1', '3'),
 ('action', 'r1', '4'),
 ('action', 'r2', '0'),
 ('action', 'r2', '1'),
 ('action', 'r2', '2'),
 ('action', 'r2', '3'),
 ('action', 'r2', '4'),
 ('action', 'speed', '0'),
 ('action', 'speed', '1'),
 ('action', 'speed', '2'),
 ('action', 'throttle', '0'),
 ('action', 'throttle', '1'),
 ('action', 'throttle', '2'),
 ('action', 'throttle', '3'),
 ('action', 'throttle', '4'),
 ('action', 'timestep', '0'),
 ('action', 'timestep', '1'),
 ('action', 'timestep', '2'),
 ('nstate', 'brake', '0'),
 ('nstate', 'brake', '1'),
 ('nstate', 'brake', '2'),
 ('nstate', 'brake', '3'),
 ('nstate', 'thrust', '0'),
 ('nstate', 'thrust', '1'),
 ('nstate', 'thrust', '2'),
 ('nstate', 'thrust', '3'),
 ('nstate', 'timestep', '0'),
 ('nstate', 'timestep', '1'),
 ('nstate', 'timestep', '2'),
 ('nstate

[('vehicle', '', ''), ('driver', '', '')]

In [72]:
# pd.Series([1, np.nan,2,None])
# pd.Series([1,np.nan,2,None,pd.NA])

In [73]:
# decoding columns
from datetime import datetime

multi_tpl = [tuple(col.split('_')) for col in df_episodes_read.columns]
# multi_tpl
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_idx
# multi_col
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i1
# i2 = multi_col.get_level_values(1).fillna('') # must be null string instead of the default pd.NA or np.nan
# i2 = [idx if isinstance(idx, int) else '' for idx in i2]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i2
# i2 = [idx if isinstance(idx, str) else '' for idx in i2]
# i2.astype('int')
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA
i3


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'rows', 'idx']
multi_col


# i2[-1]
# i2.dropna()
# i2
# l2 = ['' if i==nan else i  for i in i2]
# l2
# i2= i2[:-2].astype('int')
# i2 = i2.append(pd.Index(['','']))
# l2 = [''] + list(l2.values[:-1])
# print(l2)
# arrays = [ for i2 in l2 for i1 in l1]
# l2 = l2[:-2].astype('int') + ['', '']
# multi_col.set_levels(ll, level=['tuple','step'])
# driver_vehicle = [idx for idx in multi_idx if idx[0]=='driver' or idx[0]=='vehicle']
# multi_tpl =  [(col[0],)  if col[0]=='driver' or col[0]=='vehicle' else (col[0],int(col[1])) for col in multi_col]
# multi_idx1 = pd.MultiIndex.from_tuples(multi_idx1)
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_col

['action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'reward',
 'reward',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'vehicle',
 'driver']

['r0',
 'r0',
 'r0',
 'r0',
 'r0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r2',
 'r2',
 'r2',
 'r2',
 'r2',
 'speed',
 'speed',
 'speed',
 'throttle',
 'throttle',
 'throttle',
 'throttle',
 'throttle',
 'timestep',
 'timestep',
 'timestep',
 'brake',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'thrust',
 'timestep',
 'timestep',
 'timestep',
 'timestep',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 'timestep',
 'work',
 'brake',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'thrust',
 'timestep',
 'timestep',
 'timestep',
 'timestep',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 '',
 '']

[0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 '',
 '']

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

In [74]:
# decoding index
multi_tuple = [tuple(idx.split('_')) for idx in df_episodes_read.index]
multi_idx = pd.MultiIndex.from_tuples(multi_tuple)
# multi_idx
i0 = multi_idx.get_level_values(
    0
)  # must be null string instead of the default pd.NA or np.nan
i0 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i0
]  # convert index of level 2 type to int and '' if NA
i1 = multi_idx.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i1 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i1
]  # convert index of level 2 type to int and '' if NA
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))

multi_idx.dtypes
multi_idx
len(multi_idx)

episodestart    datetime64[ns]
timestamp       datetime64[ns]
dtype: object

MultiIndex([('2023-06-09 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.5915

25

In [75]:
df_episodes_read_multi = df_episodes_read.copy()
df_episodes_read_multi.columns = multi_col
df_episodes_read_multi.index = multi_idx
df_episodes_read_multi
df_episodes_read_multi.columns
df_episodes_read_multi.index
df_episodes_read_multi.dtypes

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   
2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   
2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
             

MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

MultiIndex([('2023-06-09 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.5915

tuple    rows      idx
action   r0        0       float64
                   1       float64
                   2       float64
                   3       float64
                   4       float64
                            ...   
state    velocity  1       float64
                   2       float64
                   3       float64
vehicle                   category
driver                    category
Length: 62, dtype: object

### recover deep multiindex from columns ['vehicle','driver']

In [76]:
df_episodes_read_multi_setindex = df_episodes_read_multi.set_index(
    ['vehicle', 'driver', df_episodes_read_multi.index]
)
df_episodes_read_multi_setindex.index
df_episodes_read_multi_setindex.columns
df_episodes_read_multi_setindex

MultiIndex([('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', '2023-06-08 09:25:10.591576', ...),
            ( 'VB7',    'wang-cheng', 

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
vehicle driver        episodestart               timestamp                           
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                                                             1   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  13.0  \
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 13:25:10.134108  13.0   
                                                 2023-06-14 14:25:10.134108  13.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  13.0   
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 13:25:10.134108  13.0   
                                                 2023-06-14 14:25:10.134108  13.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108  13.0   
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 

## append new episode to the specific parquet file

In [77]:
dfs_episode4 = dfs_episode.copy()
dfs_episode4.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(8, 'd')], level='episodestart'
)
dfs_episode4.index = dfs_episode3.index.set_levels(
    [[trucks_by_name['VB4'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode4 = dfs_episode4.swaplevel(0, 1, axis=0).swaplevel(1, 2, axis=0)
dfs_episode4.columns

# dfs_episode5 = dfs_episode.copy()
# dfs_episode5.index = dfs_episode3.index.set_levels([episodestart-pd.Timedelta(9,'d')], level='episode')
df_episodes_read_multi_setindex.columns

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

In [78]:
dfs_episodes_new = pd.concat([df_episodes_read_multi_setindex, dfs_episode4], axis=0)
dfs_episodes_new
# dfs_episodes_new.columns
# dfs_episodes_new.index

tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
vehicle driver        episodestart               timestamp                           
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB4     zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                                                             1   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  13.0  \
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 13:25:10.134108  13.0   
                                                 2023-06-14 14:25:10.134108  13.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  13.0   
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-0

#### save new dataframe back to parquet

In [79]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [80]:
dfs_episodes_new_flat = dfs_episodes_new.copy()
dfs_episodes_new_flat = dfs_episodes_new_flat.reset_index(level=['vehicle', 'driver'])
dfs_episodes_new_flat.columns
dfs_episodes_new_flat.index

MultiIndex([('vehicle',         '', ''),
            ( 'driver',         '', ''),
            ( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'a

MultiIndex([('2023-06-09 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-09 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 13:25:10.134108'),
            ('2023-06-07 09:25:10.591576', '2023-06-14 14:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 10:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 11:25:10.134108'),
            ('2023-06-08 09:25:10.591576', '2023-06-14 12:25:10.134108'),
            ('2023-06-08 09:25:10.5915

In [81]:
# encoding index
# from datetime import datetime

flat_idx = dfs_episodes_new_flat.index.to_flat_index()
flat_idx = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    if x[1] != ''
    else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in flat_idx
]
# flat_idx

# encoding columns
flat_cols = dfs_episodes_new_flat.columns.to_flat_index()
# flat_cols
# flat_cols = [
#     f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
#     if (x[1] != '' and x[3] not in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
#     if (x[1] != '' and x[3] in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
#     if (x[2] not in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}___'
#     for x in flat_cols
# ]
flat_cols = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] not in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}__{x[2]}'
    if (x[2] not in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}__'
    for x in flat_cols
]
flat_cols

['vehicle__',
 'driver__',
 'action_r0_0',
 'action_r0_1',
 'action_r0_2',
 'action_r0_3',
 'action_r0_4',
 'action_r1_0',
 'action_r1_1',
 'action_r1_2',
 'action_r1_3',
 'action_r1_4',
 'action_r2_0',
 'action_r2_1',
 'action_r2_2',
 'action_r2_3',
 'action_r2_4',
 'action_speed_0',
 'action_speed_1',
 'action_speed_2',
 'action_throttle_0',
 'action_throttle_1',
 'action_throttle_2',
 'action_throttle_3',
 'action_throttle_4',
 'action_timestep_0',
 'action_timestep_1',
 'action_timestep_2',
 'nstate_brake_0',
 'nstate_brake_1',
 'nstate_brake_2',
 'nstate_brake_3',
 'nstate_thrust_0',
 'nstate_thrust_1',
 'nstate_thrust_2',
 'nstate_thrust_3',
 'nstate_timestep_0',
 'nstate_timestep_1',
 'nstate_timestep_2',
 'nstate_timestep_3',
 'nstate_velocity_0',
 'nstate_velocity_1',
 'nstate_velocity_2',
 'nstate_velocity_3',
 'reward_timestep_0',
 'reward_work_0',
 'state_brake_0',
 'state_brake_1',
 'state_brake_2',
 'state_brake_3',
 'state_thrust_0',
 'state_thrust_1',
 'state_thrust_2',

In [82]:

dfs_episodes_new_flat.columns = flat_cols
dfs_episodes_new_flat.index = flat_idx
dfs_episodes_new_flat['vehicle__'] = dfs_episodes_new_flat['vehicle__'].astype(
    "category"
)
dfs_episodes_new_flat['driver__'] = dfs_episodes_new_flat['driver__'].astype("category")

# dfs_episodes_new_flat.index.name = 'episodestart_timestamp'
dfs_episodes_new_flat.index
dfs_episodes_new_flat.dtypes
# dfs_episodes_new_flat.columns.name = 'episodestart_timestamp'
dfs_episodes_new_flat.columns
dfs_episodes_new_flat


Index(['2023-06-09T09:25:10.591576_2023-06-14T10:25:10.134108',
       '2023-06-09T09:25:10.591576_2023-06-14T11:25:10.134108',
       '2023-06-09T09:25:10.591576_2023-06-14T12:25:10.134108',
       '2023-06-09T09:25:10.591576_2023-06-14T13:25:10.134108',
       '2023-06-09T09:25:10.591576_2023-06-14T14:25:10.134108',
       '2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108',
       '2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108',
       '2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108',
       '2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108',
       '2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108',
       '2023-06-08T09:25:10.591576_2023-06-14T10:25:10.134108',
       '2023-06-08T09:25:10.591576_2023-06-14T11:25:10.134108',
       '2023-06-08T09:25:10.591576_2023-06-14T12:25:10.134108',
       '2023-06-08T09:25:10.591576_2023-06-14T13:25:10.134108',
       '2023-06-08T09:25:10.591576_2023-06-14T14:25:10.134108',
       '2023-06-14T09:25:10.134108_2023-

vehicle__                 category
driver__                  category
action_r0_0                float64
action_r0_1                float64
action_r0_2                float64
                         ...      
state_timestep_3    datetime64[ns]
state_velocity_0           float64
state_velocity_1           float64
state_velocity_2           float64
state_velocity_3           float64
Length: 62, dtype: object

Index(['vehicle__', 'driver__', 'action_r0_0', 'action_r0_1', 'action_r0_2',
       'action_r0_3', 'action_r0_4', 'action_r1_0', 'action_r1_1',
       'action_r1_2', 'action_r1_3', 'action_r1_4', 'action_r2_0',
       'action_r2_1', 'action_r2_2', 'action_r2_3', 'action_r2_4',
       'action_speed_0', 'action_speed_1', 'action_speed_2',
       'action_throttle_0', 'action_throttle_1', 'action_throttle_2',
       'action_throttle_3', 'action_throttle_4', 'action_timestep_0',
       'action_timestep_1', 'action_timestep_2', 'nstate_brake_0',
       'nstate_brake_1', 'nstate_brake_2', 'nstate_brake_3', 'nstate_thrust_0',
       'nstate_thrust_1', 'nstate_thrust_2', 'nstate_thrust_3',
       'nstate_timestep_0', 'nstate_timestep_1', 'nstate_timestep_2',
       'nstate_timestep_3', 'nstate_velocity_0', 'nstate_velocity_1',
       'nstate_velocity_2', 'nstate_velocity_3', 'reward_timestep_0',
       'reward_work_0', 'state_brake_0', 'state_brake_1', 'state_brake_2',
       'state_brake_3', '

,vehicle__,driver__,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,...,state_thrust_2,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3
2023-06-09T09:25:10.591576_2023-06-14T10:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-09T09:25:10.591576_2023-06-14T11:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-09T09:25:10.591576_2023-06-14T12:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-09T09:25:10.591576_2023-06-14T13:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-09T09:25:10.591576_2023-06-14T14:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,MP73,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,MP73,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,MP73,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,MP73,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,MP73,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0


In [83]:
try:
    ddf_episodes_new = dd.from_pandas(dfs_episodes_new_flat, npartitions=1)
except Exception as e:
    print(e)

# dfs_episodes_new_flat


In [84]:
os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[6, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_episodes_new.to_parquet(
            'eos_dask_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

'/home/n/devel/anting_remote/eos/data'

[########################################] | 100% Completed | 101.38 ms


In [85]:
ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_episodes/_common_metadata')
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

# decoding columns index
multi_tpl = [tuple(col.split('_')) for col in df.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i1 = ['' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'rows', 'idx']
multi_col

# decoding index
multi_tuple = [tuple(idx.split('_')) for idx in df.index]
multi_idx = pd.MultiIndex.from_tuples(multi_tuple)

i0 = multi_idx.get_level_values(
    0
)  # must be null string instead of the default pd.NA or np.nan
i0 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i0
]  # convert index of level 2 type to int and '' if NA
i1 = multi_idx.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i1 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i1
]  # convert index of level 2 type to int and '' if NA
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))

multi_idx.names = ['episodestart', 'timestamp']

df.columns = multi_col
df.index = multi_idx

df = df.set_index(['vehicle', 'driver', df.index])
df
# len(multi_idx)

length: [8, 800, 8]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


MultiIndex([( 'action',       'r0',  0),
            ( 'action',       'r0',  1),
            ( 'action',       'r0',  2),
            ( 'action',       'r0',  3),
            ( 'action',       'r0',  4),
            ( 'action',       'r1',  0),
            ( 'action',       'r1',  1),
            ( 'action',       'r1',  2),
            ( 'action',       'r1',  3),
            ( 'action',       'r1',  4),
            ( 'action',       'r2',  0),
            ( 'action',       'r2',  1),
            ( 'action',       'r2',  2),
            ( 'action',       'r2',  3),
            ( 'action',       'r2',  4),
            ( 'action',    'speed',  0),
            ( 'action',    'speed',  1),
            ( 'action',    'speed',  2),
            ( 'action', 'throttle',  0),
            ( 'action', 'throttle',  1),
            ( 'action', 'throttle',  2),
            ( 'action', 'throttle',  3),
            ( 'action', 'throttle',  4),
            ( 'action', 'timestep',  0),
            ( 'a

tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
vehicle driver        episodestart               timestamp                           
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB4     zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                                                             1   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108  13.0  \
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-06-14 13:25:10.134108  13.0   
                                                 2023-06-14 14:25:10.134108  13.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  13.0   
                                                 2023-06-14 11:25:10.134108  13.0   
                                                 2023-06-14 12:25:10.134108  13.0   
                                                 2023-0

### add rows to a partition

In [86]:

ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.index
ddf.npartitions
ddf.divisions
ddf.compute()

Dask Index Structure:
npartitions=5
    object
       ...
     ...  
       ...
       ...
dtype: object
Dask Name: read-parquet-index, 2 graph layers

5

(None, None, None, None, None, None)

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__
2023-06-09T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-09T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0,MP73,zheng-longfei


In [87]:
dfs_episode5 = dfs_episode.copy()
dfs_episode5.index = dfs_episode4.index.set_levels(
    [episodestart - pd.Timedelta(9, 'd')], level='episodestart'
)
dfs_episode5.index = dfs_episode4.index.set_levels(
    [[trucks_by_name['MP74'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode5 = dfs_episode5.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode5 = dfs_episode5.reset_index(level=['vehicle', 'driver'])

dfs_episode5['vehicle'] = dfs_episode5['vehicle'].astype('category')
dfs_episode5['driver'] = dfs_episode5['driver'].astype('category')
dfs_episode5

tuple                                                 vehicle         driver   
rows                                                                           
idx                                                                            
episodestart               timestamp                                           
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108    MP74  zheng-longfei  \
                           2023-06-14 11:25:10.134108    MP74  zheng-longfei   
                           2023-06-14 12:25:10.134108    MP74  zheng-longfei   
                           2023-06-14 13:25:10.134108    MP74  zheng-longfei   
                           2023-06-14 14:25:10.134108    MP74  zheng-longfei   

tuple                                                 action               
rows                                                      r0               
idx                                                        0     1     2   
episodestart               timestamp                                       
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  13.0  14.0  \
                           2023-06-14 11:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 12:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 13:25:10.134108   12.0  13.0  14.0   
                           2023-06-14 14:25:10.134108   12.0  13.0  14.0   

tuple                                                                           
rows                                                                 r1         
idx                                                       3     4     0     1   
episodestart               timestamp                                            
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  15.0  16.0  17.0  18.0  \
                           2023-06-14 11:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 12:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 13:25:10.134108  15.0  16.0  17.0  18.0   
                           2023-06-14 14:25:10.134108  15.0  16.0  17.0  18.0   

tuple                                                        ...  state        
rows                                                         ... thrust        
idx                                                       2  ...      2    3   
episodestart               timestamp                         ...               
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108  19.0  ...    6.0  7.0  \
                           2023-06-14 11:25:10.134108  19.0  ...    6.0  7.0   
                           2023-06-14 12:25:10.134108  19.0  ...    6.0  7.0   
                           2023-06-14 13:25:10.134108  19.0  ...    6.0  7.0   
                           2023-06-14 14:25:10.134108  19.0  ...    6.0  7.0   

tuple                                                                              
rows                                                                    timestep   
idx                                                                            0   
episodestart               timestamp                                               
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108 2023-06-14 09:25:10.134108  \
                           2023-06-14 11:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 12:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 13:25:10.134108 2023-06-14 09:25:10.134108   
                           2023-06-14 14:25:10.134108 2023-06-14 09:25:10.134108   

tuple                                                                              
rows                                                                               
idx                                                                            1   
episodestart               timestamp                                               
2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134

In [88]:

dfs_episode6 = dfs_episode.copy()
dfs_episode6.index = dfs_episode6.index.set_levels(
    [episodestart - pd.Timedelta(19, 'd')], level='episodestart'
)
dfs_episode6.index = dfs_episode6.index.set_levels(
    [[trucks_by_name['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode6 = dfs_episode6.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode6 = dfs_episode6.reset_index(level=['vehicle', 'driver'])

dfs_episode6['vehicle'] = dfs_episode6['vehicle'].astype('category')
dfs_episode6['driver'] = dfs_episode6['driver'].astype('category')

# encoding columns index
dfs_episode6.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}__'
    for x in dfs_episode6.columns.to_flat_index()
]
dfs_episode6.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    if x[1] != ''
    else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in dfs_episode6.index.to_flat_index()
]
# df_episode_new.dtypes
dfs_episode6


,vehicle__,driver__,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,...,state_thrust_2,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3
2023-05-16T03:25:10.842645_2023-06-14T10:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-05-16T03:25:10.842645_2023-06-14T11:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-05-16T03:25:10.842645_2023-06-14T12:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-05-16T03:25:10.842645_2023-06-14T13:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-05-16T03:25:10.842645_2023-06-14T14:25:10.134108,MP73,wang-cheng,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0


#### encoding single dataframe of multiindex into dask single index dataframe

In [89]:
df_episode_new = dfs_episode5.copy()

# encoding columns index
df_episode_new.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}__'
    for x in dfs_episode5.columns.to_flat_index()
]
df_episode_new.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    if x[1] != ''
    else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in dfs_episode5.index.to_flat_index()
]
# df_episode_new.dtypes
df_episode_new

,vehicle__,driver__,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,...,state_thrust_2,state_thrust_3,state_timestep_0,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3
2023-06-07T09:25:10.591576_2023-06-14T10:25:10.134108,MP74,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T11:25:10.134108,MP74,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T12:25:10.134108,MP74,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T13:25:10.134108,MP74,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0
2023-06-07T09:25:10.591576_2023-06-14T14:25:10.134108,MP74,zheng-longfei,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,6.0,7.0,2023-06-14 09:25:10.134108,2023-06-14 09:25:10.154108,2023-06-14 09:25:10.174108,2023-06-14 09:25:10.194108,0.0,1.0,2.0,3.0


In [90]:

ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df.dtypes

action_r0_0          float64
action_r0_1          float64
action_r0_2          float64
action_r0_3          float64
action_r0_4          float64
                      ...   
state_velocity_1     float64
state_velocity_2     float64
state_velocity_3     float64
vehicle__           category
driver__            category
Length: 62, dtype: object

In [91]:
df_list = []
for n in range(ddf.npartitions):
    df_list.append(ddf.partitions[n])
len(df_list)

5

In [92]:
df_new_list = [dfs_episode6, df_episode_new]
for df in df_new_list:
    bHit = False
    for i, ddf in enumerate(df_list):
        # if dfs_episode_new['vehicle']
        print(f"{i}")
        # df = ddf.compute()
        # ddf.columns
        # ddf.dtypes
        vehicle = ddf['vehicle__'].compute()[0]
        driver = ddf['driver__'].compute()[0]
        # df_episode_new = df_episode_new.append(ddf)
        if df['vehicle__'][0] == vehicle and df['driver__'][0] == driver:
            bHit = True
            print(
                f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
            )
            df_list[i] = dd.concat([ddf, df], axis=0)
            # append ddf to dfs_episode_new

    if bHit == False:
        print('no hit')
        df_list.append(df_episode_new)
        # append ddf to dfs_episode_new


#     c_ep = df_episode_new['ep
#     dfs_episode_new = dfs_episode_newappend(ddf)
# print(f"splicing dfs_episode_new {c_ep} with


0
hit 0th partition
1
2
3
4
0
1
2
3
4
no hit


In [93]:

ddf_all = dd.concat(df_list, axis=0, join='outer')
# ddf_all.compute()os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[8, 800, 8],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_all.to_parquet(
            'eos_dask_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)


[########################################] | 100% Completed | 311.74 ms


In [94]:
# ddf_all.compute()
ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_dask_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in ddf.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'rows', 'idx']
df.columns = multi_col

multi_idx = [tuple(idx.split('_')) for idx in ddf.index]
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
i0 = multi_idx.get_level_values(
    0
)  # must be null string instead of the default pd.NA or np.nan
i0 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i0
]  # convert index of level 2 type to int and '' if NA
i1 = multi_idx.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i1 = [
    ''
    if str(idx) in (str(pd.NA), 'nan', '')
    else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
    for idx in i1
]  # convert index of level 2 type to int and '' if NA
multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))
df.index = multi_idx


df = df.set_index(['vehicle', 'driver', df.index])
df
df.columns
df.index
# len(multi_idx)


length: [8, 800, 8]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
vehicle driver        episodestart               timestamp                           
MP73    wang-cheng    2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-05-16 03:25:10.842645 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
MP74    zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB4     zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                          

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

MultiIndex([('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-06-09 09:25:10.591576', ...),
            ('MP73',    'wang-cheng', '2023-05-16 03:25:10.842645', ...),
            ('MP73',    'wang-cheng', '2023-05-16 03:25:10.842645', ...),
            ('MP73',    'wang-cheng', '2023-05-16 03:25:10.842645', ...),
            ('MP73',    'wang-cheng', '2023-05-16 03:25:10.842645', ...),
            ('MP73',    'wang-cheng', '2023-05-16 03:25:10.842645', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', '2023-06-07 09:25:10.591576', ...),
            ('MP73', 'zheng-longfei', 

In [95]:
df = df.sort_index()
df
df.dtypes

tuple                                                                       action   
rows                                                                            r0   
idx                                                                              0   
vehicle driver        episodestart               timestamp                           
MP73    wang-cheng    2023-05-16 03:25:10.842645 2023-06-14 10:25:10.134108   12.0  \
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-09 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
MP74    zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB4     zheng-longfei 2023-06-07 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
VB7     wang-cheng    2023-06-08 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
                      2023-06-14 09:25:10.134108 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   
        zheng-longfei 2023-06-12 09:25:10.591576 2023-06-14 10:25:10.134108   12.0   
                                                 2023-06-14 11:25:10.134108   12.0   
                                                 2023-06-14 12:25:10.134108   12.0   
                                                 2023-06-14 13:25:10.134108   12.0   
                                                 2023-06-14 14:25:10.134108   12.0   

tuple                                                                               
rows                                                                                
idx                                          

tuple   rows      idx
action  r0        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r1        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        r2        0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        speed     0             float64
                  1             float64
                  2             float64
        throttle  0             float64
                  1             float64
                  2             float64
                  3             float64
                  4             float64
        timestep  0      datetime64[ns]
                  